In [1]:
import snap
import time, gensim, pickle, json, networkx as nx, numpy as np
from networkx.readwrite import json_graph

In [2]:
with open('../REST/static/networks/twitter_users_graph2.json', 'r') as f:
    data = json.load(f)
    nxg = json_graph.node_link_graph(data, directed=True)
nxg = nxg.to_directed()

In [3]:
g = snap.PUNGraph.New()

In [4]:
for n in nxg.nodes():
    g.AddNode(n)

In [5]:
for f,t in nxg.edges():
    g.AddEdge(f, t)

In [6]:
CmtyV = snap.TCnComV()

In [7]:
start = time.time()
modularity = snap.CommunityGirvanNewman(g, CmtyV)
print(time.time()-start)

3.74089694023


In [8]:
nodes_communities = {} # {node: [community]}
for i, Cmty in enumerate(CmtyV):
    print ("Community: ")
    for NI in Cmty:
        nodes_communities.setdefault(NI, [])
        nodes_communities[NI].append(i)
        print (NI)
print ("The modularity of the network is %f" % modularity)

Community: 
32660008
105865333
216169051
225582573
234282464
246973630
267200305
338198008
430605010
450840581
631426132
728267582
784706286
821422877
859786870
888109167
924167054
965098470
968364324
Community: 
99515495
125094226
129587263
171867810
174575628
218628820
247001652
278856224
334192431
363075463
425432346
453696259
484172571
608454023
852527832
891985982
Community: 
66645671
107565051
173968248
267838228
315941679
462282747
472149195
552748176
579186200
708592708
780304124
792520470
822909215
851424847
Community: 
156001546
162427198
194739880
229523404
255993747
314344118
314449538
332799977
369836872
413849104
414291896
544378302
800981689
Community: 
39937481
99528248
252747814
306833503
331181614
363307336
400579145
435481669
469654164
492408930
724602042
938862447
Community: 
154563318
160188040
165480681
214511627
223486124
242019562
305208192
309245551
326288590
722873773
796448176
Community: 
140058875
185762586
276266430
314339417
334730050
357616346
486721124
7

In [9]:
for node in nxg.nodes():
    nxg.nodes[node]["community"] = " ".join([str(com) for com in nodes_communities[node]])

In [10]:
for ix,deg in nxg.degree(nxg.nodes()):
    nxg.node[ix]['degree'] = deg

for ix,in_deg in nxg.in_degree(nxg.nodes()):
    nxg.node[ix]['in_degree'] = in_deg
    
for ix,out_deg in nxg.out_degree(nxg.nodes()):
    nxg.node[ix]['out_degree'] = out_deg

pagerank = nx.pagerank(nxg)

for ix,v in pagerank.items():
    nxg.node[ix]["pagerank"] = v

In [11]:
def networkx_to_snappy(nxg, directed=False):
    if directed:
        g = snap.PNGraph.New()
    else:
        g = snap.PUNGraph.New()
        
    for n in nxg.nodes():
        g.AddNode(n)
    for f,t in nxg.edges():
        g.AddEdge(f, t)
        
    return g

In [12]:
data = nx.node_link_data(nxg)
data["modularity"] = modularity
data["number_of_communities"] = len(CmtyV)

snappy_directed = networkx_to_snappy(nxg, True)
snappy_undirected = networkx_to_snappy(nxg, True)

diam_directed = snap.GetBfsFullDiam(snappy_directed, 100, True)
diam_undirected = snap.GetBfsFullDiam(snappy_undirected, 100, False)

data["diameter_directed"]=diam_directed
data["diameter_undirected"]=diam_undirected
with open('../visualization/twitter_users_graph2.json', 'w') as f:
    json.dump(data, f, indent=4)
    
with open('../REST/static/networks/twitter_users_graph2.json', 'w') as f:
    json.dump(data, f, indent=4)

In [13]:
json_graph.node_link_data(json_graph.node_link_graph(data, directed=True)).keys()

['directed', 'graph', 'nodes', 'links', 'multigraph']